Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

morph = MorphAnalyzer()

%matplotlib inline

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре). 
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

In [26]:
wiki = open("./wiki_data.txt").read().split('\n')
len(wiki)

20003

In [27]:
import re


def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    normalized_tokens = [token.lower() for token in tokens if token and len(token) < 20]
    normalized_tokens = [morph.parse(token)[0].normal_form for token in normalized_tokens]
    return normalized_tokens

In [18]:
vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))
    
len(vocab)

258386

In [28]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)
len(filtered_vocab)

12442

In [29]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
    
id2word = {i:word for word, i in word2id.items()}

In [30]:
sentences = []

for text in wiki:
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [31]:
vocab_size = len(id2word)

#### skip-gram

In [32]:
def gen_batches_sg(sentences, window=5, batch_size=1000):
    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2
    
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)
                    
                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [33]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model_sg = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_sg.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
model_sg.fit(gen_batches_sg(sentences[:19000], window=8),
          validation_data=gen_batches_sg(sentences[19000:], window=8),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
         epochs=4)

Epoch 1/4
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 264s 26ms/step - accuracy: 0.7820 - loss: 0.4648 - val_accuracy: 0.7820 - val_loss: 0.5009
Epoch 2/4
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 255s 26ms/step - accuracy: 0.8242 - loss: 0.4112 - val_accuracy: 0.8246 - val_loss: 0.3994
Epoch 3/4
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 244s 24ms/step - accuracy: 0.8331 - loss: 0.3877 - val_accuracy: 0.8458 - val_loss: 0.3792
Epoch 4/4
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 283s 28ms/step - accuracy: 0.8302 - loss: 0.3959 - val_accuracy: 0.8334 - val_loss: 0.3911


In [35]:
embeddings_sg = model_sg.layers[2].get_weights()[0]

In [36]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [37]:
most_similar('кровать', embeddings_sg)

['кровать',
 'вдруг',
 'гомер',
 'джерри',
 'кирика',
 'демон',
 'идол',
 'прятаться',
 'незнакомец',
 'вставить']

In [39]:
most_similar('новость', embeddings_sg)

['новость',
 'цру',
 'веб-сайт',
 'интернет',
 'сми',
 '«наша',
 'петрик',
 'прокомментировать',
 'network',
 'харлин']

In [ ]:
print(model_sg.history.history.keys())
plt.plot(model_sg.history.history['accuracy'])
plt.plot(model_sg.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

#### cbow

In [40]:
def gen_batches_cbow(sentences, window = 8, batch_size=1000):
    left_context_length = (window/2).__ceil__() 
    right_context_length = window // 2 
    
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [43]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(8,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model_cbow = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_cbow.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
model_cbow.fit(gen_batches_cbow(sentences[:19000], window=8),
          validation_data=gen_batches_cbow(sentences[19000:], window=8),
          batch_size=1000,
          steps_per_epoch=5000,
          validation_steps=30,
         epochs=4)


Epoch 1/4
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.8176 - loss: 0.4066 - val_accuracy: 0.8779 - val_loss: 0.2978
Epoch 2/4
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 135s 27ms/step - accuracy: 0.8842 - loss: 0.2842 - val_accuracy: 0.8965 - val_loss: 0.2562
Epoch 3/4
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 132s 26ms/step - accuracy: 0.9098 - loss: 0.2253 - val_accuracy: 0.8973 - val_loss: 0.2601
Epoch 4/4
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 134s 27ms/step - accuracy: 0.9204 - loss: 0.1990 - val_accuracy: 0.9058 - val_loss: 0.2372


In [45]:
embeddings_cbow = model_cbow.layers[2].get_weights()[0]

In [46]:
most_similar('кровать', embeddings_cbow)

['кровать',
 'прятаться',
 'тарелка',
 'платье',
 'крышка',
 'мужик',
 'уголок',
 'сигарета',
 'багажник',
 'танцевать']

In [47]:
most_similar('новость', embeddings_cbow)

['новость',
 'bbc',
 'сообщить',
 'репортаж',
 'телеканал',
 'повествовать',
 'ign',
 'тв',
 'рассказать',
 'youtube']

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [48]:
import gensim

#### word2vec

In [49]:
wiki = open('wiki_data.txt').read().split('\n')
texts = [preprocess(text) for text in wiki]

In [50]:
w2v = gensim.models.Word2Vec(texts, 
                             vector_size=250, 
                             min_count=30, 
                             max_vocab_size=10000,
                             window=8,
                             epochs=5,
                             sg=1,
                             hs=1,
                             negative=6,
                             sample=1e-4,
                             ns_exponent=0.9
                             )

In [56]:
w2v.wv.most_similar("общество")

[('организация', 0.4628668427467346),
 ('общественный', 0.45965370535850525),
 ('деятельность', 0.43834418058395386),
 ('член', 0.4320813715457916),
 ('архитектурный', 0.41919445991516113),
 ('географический', 0.41482317447662354),
 ('императорский', 0.40922650694847107),
 ('петербургский', 0.40885287523269653),
 ('экономический', 0.406019002199173),
 ('почётный', 0.4043053090572357)]

In [60]:
w2v.wv.most_similar("история")

[('ни', 0.6312854290008545),
 ('раз', 0.6022486686706543),
 ('завоевать', 0.5766454935073853),
 ('один', 0.5223276615142822),
 ('медаль', 0.500001847743988),
 ('летний', 0.49543076753616333),
 ('страна', 0.492909699678421),
 ('свой', 0.47197115421295166),
 ('атланта', 0.47065240144729614),
 ('принимать', 0.4577661156654358)]

#### fasttext

In [51]:
ft = gensim.models.FastText(texts, 
                             vector_size=250, 
                             min_count=30, 
                             max_vocab_size=10000,
                             window=8,
                             epochs=5,
                             sg=1,
                             hs=1,
                             negative=6,
                             sample=1e-4,
                             ns_exponent=0.9,
                             min_n=4,
                             max_n=6
                             )

In [58]:
ft.wv.most_similar("общество")

[('общественный', 0.6214432120323181),
 ('сообщество', 0.5312342643737793),
 ('организация', 0.5034844875335693),
 ('деятельность', 0.4743858277797699),
 ('академия', 0.4558217227458954),
 ('предприниматель', 0.44683998823165894),
 ('член', 0.4461437463760376),
 ('экономический', 0.44115740060806274),
 ('почётный', 0.4265314042568207),
 ('петербургский', 0.4264426529407501)]

In [59]:
ft.wv.most_similar("история")

[('ни', 0.6313494443893433),
 ('завоевать', 0.5906491875648499),
 ('раз', 0.5655894875526428),
 ('один', 0.51510089635849),
 ('страна', 0.5041050314903259),
 ('медаль', 0.4879807233810425),
 ('принимать', 0.47962844371795654),
 ('свой', 0.466325968503952),
 ('участие', 0.4653310477733612),
 ('летний', 0.45201319456100464)]

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [61]:
data = pd.read_csv('./labeled.csv')
data['norm_text'] = data.comment.apply(preprocess)
print(len(data))
data.head()

KeyboardInterrupt: 

In [ ]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)

len(filtered_vocab)

In [ ]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [ ]:
X = []

for tokens in data['norm_text']:
    ids = [word2id[token] for token in tokens if token in word2id]
    X.append(ids)

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=100)
y = data.toxic.values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [ ]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)